# High-Fidelity Generative Image Compression

This colab can be used to compress images using HiFiC. This can also be achieved
by running `tfci.py`, as [explained in the README](https://github.com/tensorflow/compression/tree/master/models/hific#running-models-trained-by-us-locally).

Please visit [hific.github.io](https://hific.github.io) for more information.

## Setup Colab

In [ ]:
# Installs the latest version of TFC compatible with the installed TF version.
!pip install tensorflow-compression~=$(pip show tensorflow | perl -p -0777 -e 's/.*Version: (\d+\.\d+).*/\1.0/sg')

# Downloads the 'models' directory from Github.
![[ -e /tfc ]] || git clone https://github.com/tensorflow/compression /tfc
%cd /tfc/models

# Checks if tfci.py is available.
import tfci



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 5.5 MB/s eta 0:00:00
Cloning into '/tfc'...
remote: Enumerating objects: 3006, done.
remote: Counting objects: 100% (427/427), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 3006 (delta 251), reused 345 (delta 216), pack-reused 2579
Receiving objects: 100% (3006/3006), 1.40 MiB | 20.21 MiB/s, done.
Resolving deltas: 100% (1996/1996), done.
/tfc/models


## Enabling GPU

GPU should be enabled for this colab. If the next cell prints a warning, do the following:
- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down


In [ ]:
import tensorflow as tf

if not tf.config.list_physical_devices('GPU'):
  print('WARNING: No GPU found. Might be slow!')
else:
  print('Found GPU.')

Found GPU.


## Imports and Definitions

In [ ]:
import os
import zipfile
from google.colab import files
import collections
from PIL import Image
from IPython.display import Image as DisplayImage
from IPython.display import Javascript
from IPython.core.display import display, HTML
import tfci
import urllib.request

tf.get_logger().setLevel('WARN')  # Only show Warnings

FILES_DIR = '/content/files'
OUT_DIR = '/content/out'
DEFAULT_IMAGE_URL = ('https://storage.googleapis.com/hific/clic2020/'
                     'images/originals/ad249bba099568403dc6b97bc37f8d74.png')

os.makedirs(FILES_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

File = collections.namedtuple('File', ['full_path', 'num_bytes', 'bpp'])

def print_html(html):
  display(HTML(html + '<br/>'))

def make_cell_large():
  display(Javascript(
      '''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

def get_default_image(output_dir):
  output_path = os.path.join(output_dir, os.path.basename(DEFAULT_IMAGE_URL))
  print('Downloading', DEFAULT_IMAGE_URL, '\n->', output_path)
  urllib.request.urlretrieve(DEFAULT_IMAGE_URL, output_path)


## Load files

In [ ]:
#@title Setup { vertical-output: false, run: "auto", display-mode: "form" }
#@markdown #### Custom Images
#@markdown Tick the following if you want to upload your own images to compress.
#@markdown Otherwise, a default image will be used.
#@markdown
#@markdown **Note**: We support JPG and PNG (without alpha channels).
#@markdown

upload_custom_images = True #@param {type:"boolean", label:"HI"}

if upload_custom_images:
  uploaded = files.upload()
  for name, content in uploaded.items():
    with open(os.path.join(FILES_DIR, name), 'wb') as fout:
      print('Writing', name, '...')
      fout.write(content)

#@markdown #### Select a model
#@markdown Different models target different bitrates.

model = 'hific-lo' #@param ["hific-lo", "hific-mi", "hific-hi"]


Saving dynamism of a cyclist p3 final.jpg to dynamism of a cyclist p3 final.jpg
Writing dynamism of a cyclist p3 final.jpg ...


In [ ]:
if 'upload_custom_images' not in locals():
  print('ERROR: Please run the previous cell!')
  # Setting defaults anyway.
  upload_custom_images = False
  model = 'hific-lo'

In [ ]:
all_files = os.listdir(FILES_DIR)
if not upload_custom_images or not all_files:
  print('Downloading default image...')
  get_default_image(FILES_DIR)
  print()

all_files = os.listdir(FILES_DIR)
print(f'Got the following files ({len(all_files)}):')

for file_name in all_files:
  img = Image.open(os.path.join(FILES_DIR, file_name))
  w, h = img.size
  img = img.resize((w // 15, h // 15))
  print('- ' + file_name + ':')
  display(img)

Got the following files (1):
- dynamism of a cyclist p3 final.jpg:


# Compress images

In [ ]:
all_files

['dynamism of a cyclist p3 final.jpg']

In [ ]:
SUPPORTED_EXT = {'.png', '.jpg'}

all_files = os.listdir(FILES_DIR)
if not all_files:
  raise ValueError("Please upload images!")

def get_bpp(image_dimensions, num_bytes):
  w, h = image_dimensions
  return num_bytes * 8 / (w * h)

def has_alpha(img_p):
  im = Image.open(img_p)
  return im.mode == 'RGBA'

all_outputs = []
for file_name in all_files:
  if os.path.isdir(file_name):
    continue
  if not any(file_name.endswith(ext) for ext in SUPPORTED_EXT):
    print('Skipping', file_name, '...')
    continue
  full_path = os.path.join(FILES_DIR, file_name)
  if has_alpha(full_path):
    print('Skipping because of alpha channel:', file_name)
    continue
  file_name, _ = os.path.splitext(file_name)

  compressed_path = os.path.join(OUT_DIR, f'{file_name}_{model}.tfci')
  output_path = os.path.join(OUT_DIR, f'{file_name}_{model}.png')

  if os.path.isfile(output_path):
    print('Exists already:', output_path)
    num_bytes = os.path.getsize(compressed_path)
    all_outputs.append(
      File(output_path, num_bytes,
           get_bpp(Image.open(full_path).size, num_bytes)))
    continue

  print('Compressing', file_name, 'with', model, '...')
  tfci.compress(model, full_path, compressed_path)
  num_bytes = os.path.getsize(compressed_path)
  print(f'Compressed to {num_bytes} bytes.')

  print('Decompressing...')
  tfci.decompress(compressed_path, output_path)

  all_outputs.append(
      File(output_path, num_bytes,
           get_bpp(Image.open(full_path).size, num_bytes)))

print('All done!')

Compressing dynamism of a cyclist p3 final with hific-lo ...
Compressed to 6452 bytes.
Decompressing...
All done!


# Show output

In [ ]:
tfci.compress(model, full_path, compressed_path)

In [ ]:
for file_name in all_files[3:]:
  file_name, _ = os.path.splitext(file_name)
  compressed_path = os.path.join(OUT_DIR, f'{file_name}_{model}.tfci')
  output_path = os.path.join(OUT_DIR, f'{file_name}_{model}.png')
  full_path = os.path.join(FILES_DIR, file_name)
  print('Decompressing...')
  tfci.decompress(compressed_path, output_path)

  all_outputs.append(
      File(output_path, num_bytes,
           get_bpp(Image.open(full_path).size, num_bytes)))

Decompressing...


In [ ]:
make_cell_large()  # Larger output window.

for file in all_outputs:
  print_html('<hr/>')
  print(f'Showing {file.full_path} | {file.num_bytes//1000}kB | {file.bpp:.4f}bpp')
  display(Image.open(file.full_path))
  print_html('<hr/>')

### Download all compressed images.

To download all images, run the following cell.

You can also use the _Files_ tab on the left to manually select images.

---

#### **Note**: the images are saved as PNGs and thus very large. The bitrate used by HiFiC is given in the name.

In [ ]:
ZIP = '/content/images.zip'

with zipfile.ZipFile(ZIP, 'w') as zf:
  for f in all_outputs:
    path_with_bpp = f.full_path.replace('.png', f'-{f.bpp:.3f}bpp.png')
    zf.write(f.full_path, os.path.basename(path_with_bpp))

files.download(ZIP)